## Imports

In [1]:
#Commonted out the below lines as the packages are already installed in the environment

#!pip install timm
#!pip install nibabel


import nibabel as nib
import timm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



   ---------------------------------------- 0.0/3.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.3 MB 330.3 kB/s eta 0:00:10
   ---------------------------------------- 0.0/3.3 MB 262.6 kB/s eta 0:00:13
   ---------------------------------------- 0.0/3.3 MB 196.9 kB/s eta 0:00:17
    --------------------------------------- 0.1/3.3 MB 252.2 kB/s eta 0:00:13
   - -------------------------------------- 0.1/3.3 MB 350.1 kB/s eta 0:00:10
   - -------------------------------------- 0.1/3.3 MB 400.9 kB/s eta 0:00:08
   - -------------------------------------- 0.1/3.3 MB 426.7 kB/s eta 0:00:08
   -- ------------------------------------- 0.2/3.3 MB 491.5 kB/s eta 0:00:07
   -- ------------------------------------- 0.2/3.3 MB 515.5 kB/s eta 0:00:06
   ---- ----------------------------------- 0.3/3.3 MB 677.0 kB/s eta 0:00:05
   ----- ---------------------------------- 0.4/3.3 MB 771.4 kB/s eta 0:00:04
   -

c:\Users\Gelo\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available!")
else:
    print("CUDA is not available.")

CUDA is available!


In [3]:
# Set the device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

NameError: name 'model' is not defined